### -------------Yanolja crawling---------------

In [1]:
import scrapy, requests
from scrapy.http import TextResponse
import pandas as pd
import json

In [2]:
# 1. 프로젝트 생성

In [14]:
!rm -rf yanolja_hotel
!scrapy startproject yanolja_hotel

New Scrapy project 'yanolja_hotel', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/python3/notebook/project/crawling/crawling-repo-2/in_progress/yanolja_hotel

You can start your first spider with:
    cd yanolja_hotel
    scrapy genspider example example.com


In [15]:
!tree yanolja_hotel

yanolja_hotel
├── scrapy.cfg
└── yanolja_hotel
    ├── __init__.py
    ├── items.py
    ├── middlewares.py
    ├── pipelines.py
    ├── settings.py
    └── spiders
        └── __init__.py

2 directories, 7 files


In [16]:
# 2. xpath 찾기 : headers 설정

In [2]:
import requests #호텔 key값 얻는 모듈 생성 -> yanolja.py
import datetime

def get_hotel_keys():    
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36", 
           "X-Requested-With": "XMLHttpRequest"
              }
    today = datetime.date.today()
    checkin_date = today.strftime("%Y-%m-%d")
    checkout_date = (today + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
    region = "서울"
    num=0
    page = 1
    key_info = []
    while True:
        parent_url = f"https://www.yanolja.com/api/v1/contents/search?advert=KEYWORD&capacityAdult=2&capacityChild=0\
    &checkinDate={checkin_date}&checkoutDate={checkout_date}&hotel=1&keyword=서울&lat=37.50681&lng=127.06624&page={page}&limit=20&\
    searchKeyword={region}&searchType=hotel&sort=100&pathDivision=keyword-hotel&themes="
        req = requests.get(parent_url, headers=headers)
        response = req.json()
        #print('page=', str(apage))
        page += 1

        if response["motels"]["counts"] > 0:
            key = response["motels"]
            key_info.append(key)
        else:
            break
    hotel_keys = []
    for i in key_info:
        for j in i['lists']:
            hotel_keys.append(j['key'])        
    return hotel_keys

In [3]:
hotel_keys = get_hotel_keys()
len(hotel_keys)

416

### ------------xpath 확인 -> df확인 --------------

In [4]:
links = []
url = "https://www.yanolja.com/hotel/"
for hotel_key in hotel_keys:
    hotel_url = url + hotel_key
    links.append(hotel_url)
len(links)

416

In [5]:
# link 확인
link = links[0]
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"}
req = requests.get(link, headers=headers)
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 https://www.yanolja.com/hotel/3013410>

In [11]:
# xpath 확인
import json
location = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
location = json.loads(location)["props"]["pageProps"]["placeInfo"]["address"]
name = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/h1/text()')[0].extract()
level = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/div[1]/div[1]/text()')[0].extract()
score = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/div/span[1]/text()')[0].extract()
review_count = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/span/b/text()')[0].extract()
if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract() == []:
    room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
else:    
    room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract()
if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract() == []:
    room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
else:
    room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract()
price = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[4]/div/div[5]/span/text()').extract()
prices=[]
for num in range(len(room_type)):
    price = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
    price = json.loads(price)["props"]["pageProps"]["placeInfo"]["rooms"][num]["prices"][0]["price"]["discount"]["price"]
    prices.append(price)
        
location, name, level, score, review_count, room_type, room_detail, prices

('서울특별시 영등포구 경인로 870',
 '페어필드 바이 메리어트 서울',
 '4성급',
 '4.7',
 '8,797',
 ['Premier Twin',
  'Premier King',
  'Superior Single',
  'Premier Twin',
  'Premier Twin',
  'Premier King',
  'Premier King',
  'Premier Twin',
  'Premier King',
  'Premier Twin',
  'Premier King',
  'Premier Twin',
  'Premier King',
  'Premier Twin',
  'Premier King'],
 ['특가야놀자☆After 21pm Check-in/연박불가',
  '특가야놀자☆After 21pm Check-in/연박불가',
  '룸온리',
  '특가야놀자☆선착순 고층 룸UP+27시간스테이',
  '특가야놀자☆선착순 고층 룸UP',
  '특가야놀자☆선착순 고층 룸UP+25시간',
  '특가야놀자☆선착순 고층 룸UP+27시간',
  '특가야놀자☆선착순 고층 룸UP+25시간',
  '특가야놀자☆27시간스테이',
  '특가야놀자☆27시간스테이',
  '특가야놀자☆선착순 고층 룸UP',
  '특가야놀자☆Room and Parking Package',
  '특가야놀자☆Room and Parking Package',
  'stay for Breakfast',
  'stay for Breakfast'],
 ['59,900원',
  '59,900원',
  '60,000원',
  '63,000원',
  '63,000원',
  '63,000원',
  '63,000원',
  '63,000원',
  '63,000원',
  '63,000원',
  '63,000원',
  '71,800원',
  '71,800원',
  '77,300원',
  '77,300원'])

In [20]:
# df 확인
locations, names, levels, scores, review_counts, room_types, room_details, prices_1, loading_urls = [], [], [], [], [] ,[], [], [], []

num1 = 0
for link in links[:10]:
    try:
        headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"}
        req = requests.get(link, headers=headers)
        response = TextResponse(req.url, body=req.text, encoding="utf-8")

        location = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
        location = json.loads(location)["props"]["pageProps"]["placeInfo"]["address"]
        name = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/h1/text()')[0].extract()
        level = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/div[1]/div[1]/text()')[0].extract()
        score = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/div/span[1]/text()')[0].extract()
        review_count = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/span/b/text()')[0].extract()
        if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract() == []:
            room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
        else:    
            room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract()
        if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract() == []:
            room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
        else:
            room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract()
        try:
            price = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[4]/div/div[5]/span/text()').extract()
            prices=[]
            for num in range(len(room_type)):
                price = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
                price = json.loads(price)["props"]["pageProps"]["placeInfo"]["rooms"][num]["prices"][0]["price"]["discount"]["price"]
                prices.append(price)
        except:
            prices= ["예약마감"]
    except:
        location = "페이지 없음"
        location = "페이지 없음"
        name = "페이지 없음"
        level = "페이지 없음"
        score = "페이지 없음"
        review_count = "페이지 없음"
        room_type = ["페이지 없음"]
        room_detail = ["페이지 없음"]
        price = ["페이지 없음"]
        prices= ["페이지 없음"]
        
    location = [location] * len(room_type)
    name = [name] * len(room_type)
    level = [level] * len(room_type)
    score = [score] * len(room_type)
    review_count = [review_count] * len(room_type)
    link = [link] * len(room_type)
    
    locations.extend(location)
    names.extend(name)
    levels.extend(level)
    scores.extend(score)
    review_counts.extend(review_count)
    room_types.extend(room_type)
    room_details.extend(room_detail)
    prices_1.extend(prices)
    loading_urls.extend(link)
    num1+=1
    print(str(num1) ,name[0])

1 페어필드 바이 메리어트 서울
2 나인트리 프리미어 명동2
3 영등포 GMS 호텔
4 신라스테이 구로
5 노보텔 앰배서더 서울 용산
6 글래드 마포
7 신라스테이 마포
8 호텔 더 디자이너스 여의도
9 홀리데이인 익스프레스 서울 홍대
10 더 파크호텔 서울


In [21]:
df = pd.DataFrame({
    'Name': names, 'Level': levels, 'Score':scores, 'Location': locations,
    'RoomType': room_types,'Roomdetail': room_details ,'Price': prices_1, 'Link': loading_urls, 'Review' : review_counts,
})
df.tail()

,Name,Level,Score,Location,RoomType,Roomdetail,Price,Link,Review
203,더 파크호텔 서울,4성급,4.6,서울특별시 영등포구 양평로 136,파크 홈 스위트,"마스크팩 무료 증정, 대중교통 이용시 커피 무료 이용권 제공","150,000원",https://www.yanolja.com/hotel/3016365,"1,478"
204,더 파크호텔 서울,4성급,4.6,서울특별시 영등포구 양평로 136,프리미어 패밀리 스위트,"마스크팩 무료 증정, 대중교통 이용시 커피 무료 이용권 제공","150,000원",https://www.yanolja.com/hotel/3016365,"1,478"
205,더 파크호텔 서울,4성급,4.6,서울특별시 영등포구 양평로 136,파크 로얄 스위트,"마스크팩 무료 증정, 대중교통 이용시 커피 무료 이용권 제공","270,000원",https://www.yanolja.com/hotel/3016365,"1,478"
206,더 파크호텔 서울,4성급,4.6,서울특별시 영등포구 양평로 136,패밀리 디럭스 트리플+욕조,"마스크팩 무료 증정, 대중교통 이용시 커피 무료 이용권 제공",0원,https://www.yanolja.com/hotel/3016365,"1,478"
207,더 파크호텔 서울,4성급,4.6,서울특별시 영등포구 양평로 136,파크 듀얼 스위트,"마스크팩 무료 증정, 대중교통 이용시 커피 무료 이용권 제공",0원,https://www.yanolja.com/hotel/3016365,"1,478"


### --------확인 끝 spider.py로 확장---------

In [12]:
# 3. spider.py

In [20]:
%%writefile yanolja_hotel/yanolja_hotel/spiders/spider.py
import scrapy
from yanolja import get_hotel_keys
import json
import datetime

class Spider(scrapy.Spider):
    name = "YanoljaHotel"
    allow_domain = ["https://www.yanolja.com"]
    start_urls = ["https://www.yanolja.com/hotel/"]
    
    def parse(self, response):
        hotel_keys = get_hotel_keys()
        start_urls = "https://www.yanolja.com/hotel/"
        links = []
        headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"}
        for hotel_key in hotel_keys:
            hotel_url = start_urls + hotel_key
            links.append(hotel_url)
        for link in links:
            yield scrapy.Request(link, headers=headers, callback=self.parse_content)
    
    def parse_content(self, response):
        if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract() == []:
            room_types = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
        else:    
            room_types = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract()
        if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract() == []:
            room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
        else:
            room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract()
        
        roomtypes = list(zip(room_types, room_detail))

        prices=[]
        for num in range(len(room_types)):
            price = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
            price = json.loads(price)["props"]["pageProps"]["placeInfo"]["rooms"][num]["prices"][0]["price"]["discount"]["price"]
            prices.append(price)

        locations = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
        locations = json.loads(locations)["props"]["pageProps"]["placeInfo"]["address"]
        names = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/h1/text()')[0].extract()
        levels = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/div[1]/div[1]/text()')[0].extract()
        scores = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/div/span[1]/text()')[0].extract()
        review_counts = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/span/b/text()')[0].extract()
        links = response.url
        flatforms = response.url.split(".")[1]
        
        flatforms = [flatforms] * len(roomtypes)
        locations = [locations] * len(roomtypes)
        names = [names] * len(roomtypes)
        levels = [levels] * len(roomtypes)
        scores = [scores] * len(roomtypes)
        review_counts = [review_counts] * len(roomtypes)
        links = [links] * len(roomtypes)
                
        for flatform,name,level,location,score,review_count,link,room_type, price_1 in zip(flatforms, names, levels, locations, scores, review_counts, links, roomtypes, prices):
            yield {"flatform":flatform,"name":name, "level":level, "location":location, "score":score, "review_count":review_count, "roomtype":room_type, "price": price_1, "link":link}

Overwriting yanolja_hotel/yanolja_hotel/spiders/spider.py


In [21]:
# 4. settings.py : robots.txt 여부 파악 

In [22]:
# 5. 프로젝트 실행

In [23]:
%%writefile run.sh
cd yanolja_hotel
rm yanolja_hotel.csv
scrapy crawl YanoljaHotel -o yanolja_hotel.csv

Overwriting run.sh


In [24]:
!/bin/bash run.sh

2021-03-10 16:01:50 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: yanolja_hotel)
2021-03-10 16:01:50 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Jan 15 2021, 05:57:06) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1i  8 Dec 2020), cryptography 3.3.1, Platform Linux-5.4.0-1038-aws-x86_64-with-debian-buster-sid
2021-03-10 16:01:50 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-03-10 16:01:50 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'yanolja_hotel',
 'NEWSPIDER_MODULE': 'yanolja_hotel.spiders',
 'SPIDER_MODULES': ['yanolja_hotel.spiders']}
2021-03-10 16:01:50 [scrapy.extensions.telnet] INFO: Telnet Password: cbfe728fa534949b
2021-03-10 16:01:50 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy

2021-03-10 16:01:54 [urllib3.connectionpool] DEBUG: https://www.yanolja.com:443 "GET /api/v1/contents/search?advert=KEYWORD&capacityAdult=2&capacityChild=0%20%20%20%20&checkinDate=2021-03-10&checkoutDate=2021-03-11&hotel=1&keyword=%EC%84%9C%EC%9A%B8&lat=37.50681&lng=127.06624&page=12&limit=20&%20%20%20%20searchKeyword=%EC%84%9C%EC%9A%B8&searchType=hotel&sort=100&pathDivision=keyword-hotel&themes= HTTP/1.1" 200 None
2021-03-10 16:01:54 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.yanolja.com:443
2021-03-10 16:01:54 [urllib3.connectionpool] DEBUG: https://www.yanolja.com:443 "GET /api/v1/contents/search?advert=KEYWORD&capacityAdult=2&capacityChild=0%20%20%20%20&checkinDate=2021-03-10&checkoutDate=2021-03-11&hotel=1&keyword=%EC%84%9C%EC%9A%B8&lat=37.50681&lng=127.06624&page=13&limit=20&%20%20%20%20searchKeyword=%EC%84%9C%EC%9A%B8&searchType=hotel&sort=100&pathDivision=keyword-hotel&themes= HTTP/1.1" 200 None
2021-03-10 16:01:54 [urllib3.connectionpool] DEBUG: Sta

2021-03-10 16:01:56 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001558>
{'flatform': 'yanolja', 'name': '롯데시티호텔 마포', 'level': '3성급', 'location': '서울특별시 마포구 마포대로 109', 'score': '4.8', 'review_count': '1,321', 'roomtype': ('슈페리어 더블', '시티뷰, 넷플릭스+미니바'), 'price': '99,790원', 'link': 'https://www.yanolja.com/hotel/3001558'}
2021-03-10 16:01:56 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001558>
{'flatform': 'yanolja', 'name': '롯데시티호텔 마포', 'level': '3성급', 'location': '서울특별시 마포구 마포대로 109', 'score': '4.8', 'review_count': '1,321', 'roomtype': ('디럭스 트윈', '배달의민족 1만원권+체크아웃13시'), 'price': '105,330원', 'link': 'https://www.yanolja.com/hotel/3001558'}
2021-03-10 16:01:56 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001558>
{'flatform': 'yanolja', 'name': '롯데시티호텔 마포', 'level': '3성급', 'location': '서울특별시 마포구 마포대로 109', 'score': '4.8', 'review_count': '1,321', 'roomtype': ('스탠다드 더블', '배달의민족 1만원권+체크아웃13시'), 

2021-03-10 16:01:56 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3000643>
{'flatform': 'yanolja', 'name': '호텔 피제이 명동', 'level': '4성급', 'location': '서울특별시 중구 마른내로 71', 'score': '4.7', 'review_count': '1,675', 'roomtype': ('스탠다드 트윈', '특가야놀자☆'), 'price': '63,300원', 'link': 'https://www.yanolja.com/hotel/3000643'}
2021-03-10 16:01:56 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3011846>
{'flatform': 'yanolja', 'name': 'L7 강남 바이 롯데', 'level': '4성급', 'location': '서울특별시 강남구 테헤란로 415', 'score': '4.8', 'review_count': '1,492', 'roomtype': ('스탠다드 룸', '체크인시 배정'), 'price': '91,800원', 'link': 'https://www.yanolja.com/hotel/3011846'}
2021-03-10 16:01:56 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3000643>
{'flatform': 'yanolja', 'name': '호텔 피제이 명동', 'level': '4성급', 'location': '서울특별시 중구 마른내로 71', 'score': '4.7', 'review_count': '1,675', 'roomtype': ('스탠다드 더블', '특가야놀자☆'), 'price': '63,300원', 'link': 'http

2021-03-10 16:01:56 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/1000102261>
{'flatform': 'yanolja', 'name': '나인트리 프리미어 호텔 인사동', 'level': '4성급', 'location': '서울특별시 종로구 인사동길 49', 'score': '4.8', 'review_count': '1,462', 'roomtype': ('8 to 8 머무ROOM 체크인 시 배정', '숙박불가, 5객실 한정'), 'price': '60,000원', 'link': 'https://www.yanolja.com/hotel/1000102261'}
2021-03-10 16:01:56 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3000586> (referer: https://www.yanolja.com/hotel/)
2021-03-10 16:01:56 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3000643>
{'flatform': 'yanolja', 'name': '호텔 피제이 명동', 'level': '4성급', 'location': '서울특별시 중구 마른내로 71', 'score': '4.7', 'review_count': '1,675', 'roomtype': ('디럭스 트리플', '맘편한 키즈케이션 PKG'), 'price': '120,000원', 'link': 'https://www.yanolja.com/hotel/3000643'}
2021-03-10 16:01:56 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3000643>
{'flatform': 'y

...

2021-03-10 16:02:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3016365>
{'flatform': 'yanolja', 'name': '더 파크호텔 서울', 'level': '4성급', 'location': '서울특별시 영등포구 양평로 136', 'score': '4.6', 'review_count': '1,478', 'roomtype': ('나홀로 1인 초특가    +추가가능', '인원추가 및 업그레이드 가능객실, 연박시 사전문의'), 'price': '49,000원', 'link': 'https://www.yanolja.com/hotel/3016365'}
2021-03-10 16:02:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3009497> (referer: https://www.yanolja.com/hotel/)
2021-03-10 16:02:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3012800>
{'flatform': 'yanolja', 'name': '글래드 마포', 'level': '4성급', 'location': '서울특별시 마포구 마포대로 92', 'score': '4.8', 'review_count': '2,923', 'roomtype': ('내맘대로 9시간 STAY 점보 트윈', '숙박불가, 패밀리트윈'), 'price': '65,000원', 'link': 'https://www.yanolja.com/hotel/3012800'}
2021-03-10 16:02:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3012800>
{'flatfo

2021-03-10 16:02:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3016365>
{'flatform': 'yanolja', 'name': '더 파크호텔 서울', 'level': '4성급', 'location': '서울특별시 영등포구 양평로 136', 'score': '4.6', 'review_count': '1,478', 'roomtype': ('디럭스 퀸 발코니+한강뷰', '마스크팩 무료 증정, 대중교통 이용시 커피 무료 이용권 제공'), 'price': '84,000원', 'link': 'https://www.yanolja.com/hotel/3016365'}
2021-03-10 16:02:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3016365>
{'flatform': 'yanolja', 'name': '더 파크호텔 서울', 'level': '4성급', 'location': '서울특별시 영등포구 양평로 136', 'score': '4.6', 'review_count': '1,478', 'roomtype': ('패밀리 수페리어 트리플', '마스크팩 무료 증정, 대중교통 이용시 커피 무료 이용권 제공'), 'price': '94,000원', 'link': 'https://www.yanolja.com/hotel/3016365'}
2021-03-10 16:02:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3016365>
{'flatform': 'yanolja', 'name': '더 파크호텔 서울', 'level': '4성급', 'location': '서울특별시 영등포구 양평로 136', 'score': '4.6', 'review_count': '1,478', '

2021-03-10 16:02:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3015391> (referer: https://www.yanolja.com/hotel/)
2021-03-10 16:02:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3009497>
{'flatform': 'yanolja', 'name': '나인트리 프리미어 명동2', 'level': '4성급', 'location': '서울특별시 중구 마른내로 28', 'score': '4.8', 'review_count': '2,186', 'roomtype': ('8 to 8 머무ROOM 체크인 시 배정', '숙박불가, 주차불가/5객실 한정'), 'price': '52,000원', 'link': 'https://www.yanolja.com/hotel/3009497'}
2021-03-10 16:02:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3001564> (referer: https://www.yanolja.com/hotel/)
2021-03-10 16:02:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3009497>
{'flatform': 'yanolja', 'name': '나인트리 프리미어 명동2', 'level': '4성급', 'location': '서울특별시 중구 마른내로 28', 'score': '4.8', 'review_count': '2,186', 'roomtype': ('스탠다드 트윈', '야놀자단독ㅣ연박전용 Green Earth by Nine Tree, 주차불가'), 'price': '62,000

2021-03-10 16:02:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3015391>
{'flatform': 'yanolja', 'name': '노보텔 앰배서더 서울 용산', 'level': '5성급', 'location': '서울특별시 용산구 청파로20길 95', 'score': '4.7', 'review_count': '2,813', 'roomtype': ('슈페리어 트윈', '얼리버드, 취소불가'), 'price': '113,850원', 'link': 'https://www.yanolja.com/hotel/3015391'}
2021-03-10 16:02:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3015391>
{'flatform': 'yanolja', 'name': '노보텔 앰배서더 서울 용산', 'level': '5성급', 'location': '서울특별시 용산구 청파로20길 95', 'score': '4.7', 'review_count': '2,813', 'roomtype': ('슈페리어 더블', '얼리버드, 취소불가'), 'price': '124,850원', 'link': 'https://www.yanolja.com/hotel/3015391'}
2021-03-10 16:02:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3015391>
{'flatform': 'yanolja', 'name': '노보텔 앰배서더 서울 용산', 'level': '5성급', 'location': '서울특별시 용산구 청파로20길 95', 'score': '4.7', 'review_count': '2,813', 'roomtype': ('슈페리어 트윈', '체크인 18시 체크아웃 

2021-03-10 16:02:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001564>
{'flatform': 'yanolja', 'name': '신라스테이 구로', 'level': '3성급', 'location': '서울특별시 동작구 시흥대로 596', 'score': '4.6', 'review_count': '4,113', 'roomtype': ('스탠다드 트윈', 'TO-GO 모닝 도시락 2인 패키지'), 'price': '76,230원', 'link': 'https://www.yanolja.com/hotel/3001564'}
2021-03-10 16:02:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001564>
{'flatform': 'yanolja', 'name': '신라스테이 구로', 'level': '3성급', 'location': '서울특별시 동작구 시흥대로 596', 'score': '4.6', 'review_count': '4,113', 'roomtype': ('스탠다드 더블', '단독★24시간STAY+조식1+1'), 'price': '78,650원', 'link': 'https://www.yanolja.com/hotel/3001564'}
2021-03-10 16:02:18 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001564>
{'flatform': 'yanolja', 'name': '신라스테이 구로', 'level': '3성급', 'location': '서울특별시 동작구 시흥대로 596', 'score': '4.6', 'review_count': '4,113', 'roomtype': ('스탠다드 더블', 'TO-GO 모닝 도시락 2인 패키지'

In [25]:
data = pd.read_csv('./yanolja_hotel/yanolja_hotel.csv')

In [26]:
data

,flatform,name,level,location,score,review_count,roomtype,price,link
0,yanolja,롯데시티호텔 마포,3성급,서울특별시 마포구 마포대로 109,4.8,"1,321","스탠다드 더블,선착순 한정, 18시 체크인","74,840원",https://www.yanolja.com/hotel/3001558
1,yanolja,롯데시티호텔 마포,3성급,서울특별시 마포구 마포대로 109,4.8,"1,321","체크인 시 객실 배정,기준 2명 / 최대 2명","79,000원",https://www.yanolja.com/hotel/3001558
2,yanolja,롯데시티호텔 마포,3성급,서울특별시 마포구 마포대로 109,4.8,"1,321","스탠다드 더블,리뷰이벤트 l 13시 체크아웃","83,160원",https://www.yanolja.com/hotel/3001558
3,yanolja,롯데시티호텔 마포,3성급,서울특별시 마포구 마포대로 109,4.8,"1,321","스탠다드 더블,기준 2명 / 최대 2명","83,160원",https://www.yanolja.com/hotel/3001558
4,yanolja,롯데시티호텔 마포,3성급,서울특별시 마포구 마포대로 109,4.8,"1,321","스탠다드 더블,야놀자단독ㅣ나만의 24시간 STAY PKG","91,470원",https://www.yanolja.com/hotel/3001558
5,yanolja,롯데시티호텔 마포,3성급,서울특별시 마포구 마포대로 109,4.8,"1,321","스탠다드 더블룸 –,★30시간 STAY, 21시 체크아웃","91,470원",https://www.yanolja.com/hotel/3001558
6,yanolja,롯데시티호텔 마포,3성급,서울특별시 마포구 마포대로 109,4.8,"1,321","슈페리어 더블,시티뷰, 넷플릭스+미니바","99,790원",https://www.yanolja.com/hotel/3001558
7,yanolja,롯데시티호텔 마포,3성급,서울특별시 마포구 마포대로 109,4.8,"1,321","디럭스 트윈,배달의민족 1만원권+체크아웃13시","105,330원",https://www.yanolja.com/hotel/3001558
8,yanolja,롯데시티호텔 마포,3성급,서울특별시 마포구 마포대로 109,4.8,"1,321","스탠다드 더블,배달의민족 1만원권+체크아웃13시","105,330원",https://www.yanolja.com/hotel/3001558
9,yanolja,롯데시티호텔 마포,3성급,서울특별시 마포구 마포대로 109,4.8,"1,321","디럭스 트윈,기준 2명 / 최대 2명","105,330원",https://www.yanolja.com/hotel/3001558


In [ ]:
# 6. pipeline.py : 크롤링한 데이터를 mysql에 저장

# -----------TEST------------